# December 21, 2024

https://adventofcode.com/2024/day/21

In [1]:
# abstract base classes
import abc

In [2]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [4]:
test_str = f'''029A
980A
179A
456A
379A'''
test = [x.strip() for x in test_str.split("\n")]

In [5]:
fn = "../data/2024/21.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz = [x.strip() for x in text]

Numeric:
```
+---+---+---+  
| 7 | 8 | 9 |  
+---+---+---+  
| 4 | 5 | 6 |  
+---+---+---+  
| 1 | 2 | 3 |  
+---+---+---+  
    | 0 | A |  
    +---+---+  
```

Directional
```
    +---+---+  
    | ^ | A |  
+---+---+---+  
| < | v | > |  
+---+---+---+  
```

# Part 1

In [6]:
class Robot(metaclass=abc.ABCMeta):
# directional

    @abc.abstractmethod
    def __init__(self, keypad=None):
        self.keypad = keypad

    def paths(self, btn0, btn1):
        '''find all relevant paths from btn0 to btn1'''
        # assumes that the gap is in a corner
        # there's no use mixing up horizontals and verticals because it will make the next level up more expensive

        # first find x and y locs
        y0, x0 = self.keypad[btn0]
        y1, x1 = self.keypad[btn1]
        ygap, xgap = self.keypad["X"]

        # figure out the horizontal and vertical moves needed
        dx = x0 - x1
        dy = y0 - y1

        if dx < 0:
            hor = '>' * abs(dx)
        else:
            hor = '<' * abs(dx)
        if dy < 0:
            ver = 'v' * abs(dy)
        else:
            ver = '^' * abs(dy)

        # if staying same row or same col, then only one type of move needed
        if dx == 0:
            return [ver]
        if dy == 0:
           return [hor]

        # otherwise, determine if the gap forces a particular path
        if ygap == y0 and xgap == x1:
            paths = [ver + hor]
        elif ygap == y1 and xgap == x0:
            paths = [hor + ver]
        # gap won't get in way, so we can go ver then hor or vice versa
        # prioritize ^ or >, then v, then < to be last, since they are closer to the A button for the supervising robot
        else:
            #paths = [ver + hor, hor + ver]
            if dy > 0: # ^
                if dx < 0: # ^ and > which tie, so it's arbitrary
                    paths = [ver + hor]#, hor + ver]
                else: # < before v
                    paths = [hor + ver]
            else: # v
                if dx < 0: # > after v
                    paths = [ver + hor]
                else: # < before v
                    paths = [hor + ver]


        return paths
    
    def len_seq_path( self, seq ):
        all_paths = self.seq_path( seq )
        return min( [len(x) for x in all_paths] )
    

class Dirbot(Robot):
    def __init__(self):
        #keypad = [['7','8','9'],['4','5','6'],['1','2','3'],['X', '0', 'A']]
        keypad = {
            'X': [0,0],
            '^': [0,1],
            'A': [0,2],
            '<': [1,0],
            'v': [1,1],
            '>': [1,2]
        }
        
        super(Dirbot, self).__init__(keypad)

    def seq_path( self, seq ):
        child_paths = self.child.seq_path( seq )

        all_paths = list()
        for cp in child_paths:
            seq = 'A' + cp
        
            parts = list()
            for i in range(len(seq)-1):
                paths = self.paths(seq[i], seq[i+1])
                for i, p in enumerate(paths):
                    paths[i] = p + 'A'
                parts.append(paths)


            so_far = parts[-1]
            for p in parts[-2::-1]:
                iter = list()
                for seq in p:
                    iter += [seq + x for x in so_far]
                so_far = iter

            all_paths += so_far

        # return only the shortest paths
        best_len = 9e99
        best_paths = list()
        for p in all_paths:
            #if self.__prio__(p) < best_len:
            if len(p) < best_len:
                best_len = len(p)
                best_paths = [p]
            #elif self.__prio__(p) == best_len:
            elif len(p) == best_len:
                best_paths.append(p)

        dprint(len(best_paths))

        return best_paths




class Numbot(Robot):
    def __init__(self):
        keypad = {
            '7': [0,0],
            '8': [0,1],
            '9': [0,2],
            '4': [1,0],
            '5': [1,1],
            '6': [1,2],
            '1': [2,0],
            '2': [2,1],
            '3': [2,2],
            'X': [3,0],
            '0': [3,1],
            'A': [3,2]
        }
        super(Numbot, self).__init__(keypad)

    def seq_path( self, seq ):
        seq = 'A' + seq
        parts = list()
        for i in range(len(seq)-1):
            paths = self.paths(seq[i], seq[i+1])
            for i, p in enumerate(paths):
                paths[i] = p + 'A'
            parts.append(paths)


        so_far = parts[-1]
        for p in parts[-2::-1]:
            iter = list()
            for seq in p:
                iter += [seq + x for x in so_far]
            so_far = iter

        return so_far

In [7]:
def part1( puzz ):
    numbot = Numbot()
    dirbot1 = Dirbot()
    dirbot1.child = numbot
    dirbot2 = Dirbot()
    dirbot2.child = dirbot1

    tot = 0
    lengths = list()
    for seq in puzz:
        dprint("----------")
        dprint(seq)
        slen = dirbot2.len_seq_path(seq)
        lengths.append(slen)
        num = int(seq[:-1])
        tot += slen*num

    return tot, lengths


In [9]:
# best len, prioritze < before v before ^/>
DEBUG = True
part1( test )

----------
029A
1
1
----------
980A
1
1
----------
179A
1
1
----------
456A
1
1
----------
379A
1
1


(126384, [68, 60, 68, 64, 64])

In [10]:
# best len, prioritze < before v before ^ before >
DEBUG = True
part1( puzz )

----------
540A
1
1
----------
582A
1
1
----------
169A
1
1
----------
593A
1
1
----------
579A
1
1


(176870, [72, 68, 76, 74, 72])

# Part2

In [12]:
def part2( puzz, num_dirbots = 25 ):
    #dbone = Dirbot()
    #dbone.child = Numbot()
    dblist = [ Dirbot() ]
    dblist[0].child = Numbot()
    for i in range(1, num_dirbots):
        dblist.append( Dirbot() )
        dblist[i].child = dblist[i-1]
        #dbtwo.child = dbone
        #dbone = dbtwo

    tot = 0
    lengths = list()
    for seq in puzz:
        dprint("----------")
        print(seq)
        slen = dblist[-1].len_seq_path(seq)
        lengths.append(slen)
        num = int(seq[:-1])
        tot += slen*num

    return tot, lengths


This method takes too long!

# Part2 Redux:
### Memoize

In [14]:
def numpad( btn0, btn1 ):
    '''return the movement sequence for the numbot to get from btn0 to btn1'''
    keypad = {
        '7': [0,0],
        '8': [0,1],
        '9': [0,2],
        '4': [1,0],
        '5': [1,1],
        '6': [1,2],
        '1': [2,0],
        '2': [2,1],
        '3': [2,2],
        'X': [3,0],
        '0': [3,1],
        'A': [3,2]
    }

    # first find x and y locs
    y0, x0 = keypad[btn0]
    y1, x1 = keypad[btn1]
    ygap, xgap = keypad["X"]

    # figure out the horizontal and vertical moves needed
    dx = x0 - x1
    dy = y0 - y1

    if dx < 0:
        hor = '>' * abs(dx)
    else:
        hor = '<' * abs(dx)
    if dy < 0:
        ver = 'v' * abs(dy)
    else:
        ver = '^' * abs(dy)

    # if staying same row or same col, then only one type of move needed
    if dx == 0:
        return [ver]
    if dy == 0:
        return [hor]

    # otherwise, determine if the gap forces a particular path
    if ygap == y0 and xgap == x1:
        paths = [ver + hor]
    elif ygap == y1 and xgap == x0:
        paths = [hor + ver]
    # gap won't get in way, so we can go ver then hor or vice versa
    # prioritize ^ or >, then v, then < to be last, since they are closer to the A button for the supervising robot
    else:
        #paths = [ver + hor, hor + ver]
        if dy > 0: # ^
            if dx < 0: # ^ and > which tie
                paths = [ver + hor]#, hor + ver]
            else: # < before v
                paths = [hor + ver]
        else: # v
            if dx < 0: # > after v
                paths = [ver + hor]
            else: # < before v
                paths = [hor + ver]
    return paths
    

In [15]:
def seq_len( seq, n, memo=dict() ):
    '''return the number of button presses to enter the given seq
    with n levels of indirection'''
    if n in memo.keys() and seq in memo[n]:
        return memo[n][seq]
    
    if n == 0:
        return len(seq)
    
    move_map = {
        'A': { 'A': "", '^': "<", '<': "v<<", 'v': "<v", '>': "v" },
        '^': { 'A': ">", '^': "", '<': "v<", 'v': "v", '>': "v>" },
        '<': { 'A': ">^^", '^': ">^", '<': "", 'v': ">", '>': ">>" },
        'v': { 'A': "^>", '^': "^", '<': "<", 'v': "", '>': ">" },
        '>': { 'A': "^", '^': "<^", '<': "<<", 'v': "<", '>': "" }
    }
    
    # finger starts over A button
    seq = "A" + seq
    tot = 0

    # for each button press, the supervisor needs to follow the sequence in move_map then press A
    for i in range(len(seq)-1):
        btn0, btn1 = seq[i], seq[i+1]
        tot += seq_len( move_map[ btn0 ][ btn1 ] + "A", n-1, memo )

    # save results
    if n not in memo.keys():
        memo[n] = dict()
    memo[n][seq[1:]] = tot

    return tot

In [16]:
def complexity( puzz, ndirbots = 2 ):
    ans = 0
    lengths = list()
    for p in puzz:
        seq = "A" + p
        slen = 0
        for i in range(len(seq)-1):
            btn0 = seq[i]
            btn1 = seq[i+1]
            mv = numpad(btn0, btn1)[0]
            slen +=  seq_len( mv + "A", ndirbots )

        lengths.append(slen)
        ans += int(p[:-1]) * slen
    return ans, lengths

    



In [17]:
debug = False
complexity( test )

(126384, [68, 60, 68, 64, 64])

In [18]:
complexity( puzz )

(176870, [72, 68, 76, 74, 72])

In [19]:
complexity(puzz, 25)

(223902935165512,
 [91906526790, 86475783008, 91059074548, 93831469524, 91387668328])